### Setup

In [191]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [192]:
import cv2 as cv
import numpy as np
from datetime import datetime

from utils import get_coordinates, crop_and_mask_image

### Read the Video File

In [193]:
dir_project = "C:/Users/eats/projects/IASS"

name_video = "jpo_tunjungan_02"
side = "right"
path_video = f"asset/{name_video}.mp4"
path_video = os.path.join(dir_project, path_video)
i_skip = 1
name_datetime = str(datetime.now().date()).replace("-", "")
name_dataset = f"{name_datetime}-{name_video}_{side}-skip_{i_skip}"

dir_output = os.path.join(dir_project, "asset", "dataset", name_dataset)
os.makedirs(dir_output, exist_ok=True)

# get one image sample
cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)
success, imx = cap.read()

# re-read
cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)

### Define Region Coordinates

In [194]:
def adjust_xy(polygon: np.ndarray, x_min: int, y_min: int) -> np.ndarray:
    return np.array([(max(0, x - x_min), max(0, y - y_min)) for (x, y) in polygon])


def adjust_site_region(
    polygon: list | np.ndarray, 
    line_in: list | np.ndarray | None = None, 
    line_out: list | np.ndarray | None = None,
) -> tuple[
    tuple[int, int, int, int], 
    np.ndarray, 
    np.ndarray | None, 
    np.ndarray | None,
]:
    # type validity check
    if isinstance(polygon, list):
        polygon = np.array(polygon)
    if isinstance(line_in, list):
        line_in = np.array(line_in)
    if isinstance(line_out, list):
        line_out = np.array(line_out)

    # shape validity check
    if polygon.shape != (4, 2):
        raise ValueError(f"{polygon.shape} is invalid shape for `polygon`")
    
    # get min and max pixel value from polygon
    x_min, y_min = np.min(polygon, axis=0).tolist()
    x_max, y_max = np.max(polygon, axis=0).tolist()
    # adjust original polygon and line towards crop image
    polygon = adjust_xy(polygon=polygon, x_min=x_min, y_min=y_min)
    if not line_in is None:
        if line_in.shape != (2, 2):
            raise ValueError(f"{line_in.shape} is invalid shape for `line_in`")
        line_in = adjust_xy(line_in, x_min=x_min, y_min=y_min)
    if not line_out is None:
        if line_out.shape != (2, 2):
            raise ValueError(f"{line_out.shape} is invalid shape for `line_out`")
        line_out = adjust_xy(line_out, x_min=x_min, y_min=y_min)
    return (x_min, y_min, x_max, y_max), polygon, line_in, line_out



In [ ]:
# # # # jpo_swisbel_02_right.mp4
# line_in = None
# line_out = [[30, 733], [1868, 725]]
# polygon = [[165, 1079], [791, 166], [1178, 166], [1868, 1079]]
# (x_min, y_min, x_max, y_max), polygon, line_in, line_out = adjust_site_region(polygon=polygon, line_in=line_in, line_out=line_out)

# # # jpo_tunjugnan_01_right.mp4
line_in = None
line_out = [[42, 989], [1873, 996]]
polygon = [[516, 1079], [876, 726], [1453, 774], [1648, 1079]]
(x_min, y_min, x_max, y_max), polygon, line_in, line_out = adjust_site_region(polygon=polygon, line_in=line_in, line_out=line_out)

In [196]:
(x_min, y_min, x_max, y_max), polygon, line_in, line_out

((516, 726, 1648, 1079),
 array([[   0,  353],
        [ 360,    0],
        [ 937,   48],
        [1132,  353]]),
 None,
 array([[   0,  263],
        [1357,  270]]))

Original Image

In [197]:
# # Buat window dan set event callback
# cv.namedWindow("Image", cv.WINDOW_NORMAL)
# cv.namedWindow("Image")
# cv.setMouseCallback("Image", get_coordinates)
# cv.imshow("Image", imx)
# cv.waitKey(0)

Crop Image

In [198]:
# imx_crop = imx[y_min:y_max, x_min:x_max]
# imx_crop = crop_and_mask_image(
#     img=imx, 
#     x_min=x_min, 
#     x_max=x_max, 
#     y_min=y_min, 
#     y_max=y_max, 
#     polygon=polygon,
# )
# cv.drawContours(imx_crop, [line_out], -1, (0, 0, 255), thickness=2)
# print(imx_crop.shape)

# cv.namedWindow("Image", cv.WINDOW_NORMAL)
# cv.namedWindow("Image")
# cv.setMouseCallback("Image", get_coordinates)
# cv.imshow("Image", imx_crop)
# cv.waitKey(0)

### Extract Each Frame of Video to JPG

In [199]:
# get one image sample
cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)
success, imx = cap.read()

# re-read
cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)

In [200]:
idx_img = 0
i = i_skip
cv.namedWindow("Video", cv.WINDOW_NORMAL)
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    if i == 0:
        i = i_skip
        continue
    
    # # imm = np.zeros_like(im0)
    # # imm[y_min:y_max, x_min:x_max] = im0[y_min:y_max, x_min:x_max]
    # imm = im0[y_min:y_max, x_min:x_max]
    
    # crop image
    imm = im0[y_min:y_max, x_min:x_max]

    # Create a black mask (same height & width as image, single channel)
    mask = np.zeros(imm.shape[:2], dtype=np.uint8)

    # Fill the polygon area with white (255)
    cv.fillPoly(mask, [polygon], color=255)

    # Apply the mask to keep only the polygon area
    imm = cv.bitwise_and(imm, imm, mask=mask)
    
    cv.imshow("Video", imm)
    if cv.waitKey(1) & 0xFF == ord("q"):
        break
    
    # video_writer.write(imm)
    
    path_output = os.path.join(dir_output, f'{name_video}_{idx_img}.jpg')
    cv.imwrite(path_output, imm)
    
    i -= 1
    idx_img += 1
    
    # break

cap.release()
# video_writer.release()
cv.destroyAllWindows()

Video frame is empty or video processing has been successfully completed.


In [201]:
# os.path.join(dir_output, f'{name_video}_{idx_img}.jpg').replace("/", "\\")

In [202]:
# cv.waitKey(0)
# cap.release()
# cv.destroyAllWindows()